# Imports

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import sys

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import random
import math

import seaborn as sns

from IPython.display import FileLink, FileLinks


In [2]:
!tree "glioblastoma_data/raw_data/2021_lea_renaud_internship/reorganized/sox"

glioblastoma_data/raw_data/2021_lea_renaud_internship/reorganized/sox
├── N1
│   ├── CT
│   │   ├── 0Gy
│   │   │   ├── 20210305u251.nd2crisprCT0GYp1.txt
│   │   │   ├── 20210305u251.nd2crisprCT0GYp2.txt
│   │   │   ├── 20210305u251.nd2crisprCT0GYtotal.csv
│   │   │   ├── 20210305u251.nd2crisprCT0GYtotal.svg
│   │   │   ├── 20210305u251.nd2crisprCT0GYtotal.txt
│   │   │   └── 20210305u251.nd2crisprCT0GYtotal.xlsx
│   │   └── 4Gy
│   │       ├── 20210305u251.nd2crisprCT4GYp1.txt
│   │       ├── 20210305u251.nd2crisprCT4GYp2.txt
│   │       ├── 20210305u251.nd2crisprCT4GYp3.txt
│   │       ├── 20210305u251.nd2crisprCT4GYp4.txt
│   │       ├── 20210305u251.nd2crisprCT4GYtotal.csv
│   │       ├── 20210305u251.nd2crisprCT4GYtotal.svg
│   │       ├── 20210305u251.nd2crisprCT4GYtotal.txt
│   │       └── 20210305u251.nd2crisprCT4GYtotal.xlsx
│   └── KO
│       ├── 0Gy
│       │   ├── 050321.nd2u251crisprsox20GY1.txt
│       │   ├── 050321.nd2u251crisprsox20GY2.txt
│       │   ├── 050321.nd2u25

# Constants

In [3]:
INPUT_DATA_FOLDER = "glioblastoma_data/raw_data/2021_lea_renaud_internship/reorganized"
OUTPUT_TRANSCRIPTIONS_TXT_FOLDER = "videos_transcriptions_txt"
OUTPUT_TRANSCRIPTIONS_CSV_FOLDER = "videos_transcriptions_csv"
EXAMPLE_TRANSCRIPTION_TXT_FILE = "videos_transcriptions_txt/normal/n2/2021n210Gy2B.txt"

BAD_FILES = ["160930.nd2u2510gyp2", "total"]#, "crispr"]

In [4]:
# Output folder preparation
!rm -rf "videos_transcriptions_txt"
!rm -rf "videos_transcriptions_csv"

#!mkdir -p "videos_transcriptions_txt"

#!mkdir -p "videos_transcriptions_txt/normal"
#!mkdir -p "videos_transcriptions_txt/sox2"

!mkdir -p "videos_transcriptions_txt/normal/n1"
!mkdir -p "videos_transcriptions_txt/normal/n2"
!mkdir -p "videos_transcriptions_txt/normal/n3"

#!mkdir -p "videos_transcriptions_txt/sox2/n1"
#!mkdir -p "videos_transcriptions_txt/sox2/n2"

!mkdir -p "videos_transcriptions_txt/sox/n1/ct"
!mkdir -p "videos_transcriptions_txt/sox/n1/ko"
!mkdir -p "videos_transcriptions_txt/sox/n2/ct"
!mkdir -p "videos_transcriptions_txt/sox/n2/ko"

#!mkdir -p "videos_transcriptions_txt/sox2/n1/ct"
#!mkdir -p "videos_transcriptions_txt/sox2/n2/ko"

#!mkdir -p "videos_transcriptions_csv"

!mkdir -p "videos_transcriptions_csv/normal/n1"
!mkdir -p "videos_transcriptions_csv/normal/n2"
!mkdir -p "videos_transcriptions_csv/normal/n3"

!mkdir -p "videos_transcriptions_csv/sox/n1/ct"
!mkdir -p "videos_transcriptions_csv/sox/n1/ko"
!mkdir -p "videos_transcriptions_csv/sox/n2/ct"
!mkdir -p "videos_transcriptions_csv/sox/n2/ko"

#!tree

# Hyper-parameters

In [5]:
DEBUG = True

In [6]:
if DEBUG:
    for dirname, _, filenames in os.walk(INPUT_DATA_FOLDER):
        for filename in filenames:
            print(os.path.join(dirname, filename))

glioblastoma_data/raw_data/2021_lea_renaud_internship/reorganized/normal/8Gy/8gy n3/20211111u2518GyA3B.svg
glioblastoma_data/raw_data/2021_lea_renaud_internship/reorganized/normal/8Gy/8gy n3/20211111u2518GyB3B.csv
glioblastoma_data/raw_data/2021_lea_renaud_internship/reorganized/normal/8Gy/8gy n3/20211111u2518GyB4C.csv
glioblastoma_data/raw_data/2021_lea_renaud_internship/reorganized/normal/8Gy/8gy n3/20211111u2518GyC3B.png
glioblastoma_data/raw_data/2021_lea_renaud_internship/reorganized/normal/8Gy/8gy n3/20211111u2518GyA4B.png
glioblastoma_data/raw_data/2021_lea_renaud_internship/reorganized/normal/8Gy/8gy n3/20211111u2518GyA3B.csv
glioblastoma_data/raw_data/2021_lea_renaud_internship/reorganized/normal/8Gy/8gy n3/20211111u2518GyA4A.png
glioblastoma_data/raw_data/2021_lea_renaud_internship/reorganized/normal/8Gy/8gy n3/20211111u2518GyB3B.png
glioblastoma_data/raw_data/2021_lea_renaud_internship/reorganized/normal/8Gy/8gy n3/20211111u2518GyB4C.svg
glioblastoma_data/raw_data/2021_lea_r

# 1) Extraction of txt manual transcriptions

In [7]:
import shutil

# Search for transcription files (.txt files)
for dirname, _, filenames in os.walk(INPUT_DATA_FOLDER):
    for filename in filenames:
        # Extract experiement informations
        exp_folder = dirname[len(INPUT_DATA_FOLDER):]
        #print(exp_folder)
        exp_folder = exp_folder.lower()
        
        # Experiement type (normal/sox)
        experiement_type = None
        if "normal" in exp_folder:
            experiement_type = "normal"
        if "sox" in exp_folder:
            experiement_type = "sox"
        
        # Experiement group (n1/n2/...)
        experiement_group = None
        if "n1" in exp_folder:
            experiement_group = "n1"
        if "n2" in exp_folder:
            experiement_group = "n2"
        if "n3" in exp_folder:
            experiement_group = "n3"
            
        # Sox2 sub folders (ct/ko)
        if experiement_type == "sox":
            if "ct" in exp_folder:
                experiement_group += "/ct"
            if "ko" in exp_folder or "crispr" in exp_folder:
                experiement_group += "/ko"
            
        if experiement_type == None:
            if DEBUG: print("Not valid folder found (expected normal or sox in path) ignored file:", os.path.join(dirname, filename))
            continue
            
        if experiement_group == None:
            if DEBUG: print("Not valid folder found (expected n1 or n2 or n3 in path) ignored file:", os.path.join(dirname, filename))
            continue
            
        file_path = os.path.join(dirname, filename)
        valid_file = False
        
        if filename[-4:] == ".txt":
            valid_file = True
            for i in BAD_FILES:
                if i in filename.lower():
                    valid_file = False
                    break
            if valid_file:
                if DEBUG: print("Valid file found:",filename)
                shutil.copyfile(file_path, OUTPUT_TRANSCRIPTIONS_TXT_FOLDER+"/"+experiement_type+"/"+experiement_group+"/"+filename)
!zip -r "videos_transcriptions_txt.zip" videos_transcriptions_txt

Valid file found: 2021 11 11  8Gy B3A.txt
Valid file found: 2021 11 11 8Gy B4B.txt
Valid file found: 2021 11 11 8Gy B3B.txt
Valid file found: 2021 11 11  8Gy C3B.txt
Valid file found: 2021 11 11 8Gy B4A.txt
Valid file found: 2021 11 11 8Gy C3C.txt
Valid file found: 2021 11 11 8Gy A3B.txt
Valid file found: 2021 11 11 8Gy B4C.txt
Valid file found: 2021 11 11 8Gy A4C.txt
Valid file found: 2021 11 11 8Gy C3A.txt
Valid file found: 2021 11 11 8Gy C4C.txt
Valid file found: 2021 11 11 8Gy A4A.txt
Valid file found: 2021 11 11 8Gy A3A.txt
Valid file found: 2021 11 11 8Gy A4B.txt
Valid file found: 2021 11 11 8Gy B3C.txt
Valid file found: 2021 11 11 8Gy C4A.txt
Valid file found: 2021 11 11 8Gy A3C.txt
Valid file found: 160930.nd2u2518gyp4.txt
Valid file found: 160930.nd2u2518gyp2.txt
Valid file found: 160930.nd2u2518gyp1.txt
Valid file found: 160930.nd2u2518gyp3.txt
Valid file found: 2020 04 11 B3B  8gy.txt
Valid file found: 2020 04 11 C4A  8gy.txt
Valid file found: 2020 04 11 C3A  8gy.txt
Valid f

In [8]:
!tree

.
├── glioblastoma_data
│   ├── extracted_data
│   │   └── cell_behaviors
│   │       ├── examples
│   │       │   ├── example_videos_transcriptions_csv.csv
│   │       │   └── example_videos_transcriptions_txt.txt
│   │       ├── README.txt
│   │       ├── videos_transcriptions_csv
│   │       │   ├── normal
│   │       │   │   ├── n1
│   │       │   │   │   ├── 0gy
│   │       │   │   │   │   ├── normal_[n1]_0gy_201104.nd2u2510Gy1A.csv
│   │       │   │   │   │   ├── normal_[n1]_0gy_201104.nd2u2510GyA3A.csv
│   │       │   │   │   │   ├── normal_[n1]_0gy_2020 11 04 0Gy 1A.csv
│   │       │   │   │   │   ├── normal_[n1]_0gy_2020 11 04 0Gy A2B.csv
│   │       │   │   │   │   ├── normal_[n1]_0gy_2020 11 04 0Gy A3A.csv
│   │       │   │   │   │   └── normal_[n1]_0gy_2020 11 04 .0Gy B2A.csv
│   │       │   │   │   ├── 10gy
│   │       │   │   │   │   ├── normal_[n1]_10gy_160930.nd2u25110gyp1.csv
│   │       │   │   │   │   ├── normal_[n1]_10gy_160930.nd2u25110gyp2.csv
│   │       │   │   

# 2) Convert txt transcription to formated CSV files

In [9]:
def convert_to_dataframe(file_path):
    data = []
    radiation_level = -1
    file_path_split = file_path.split("/")
    file_name = file_path_split[-1]
    
    if "normal" in file_path_split:
        experiement_type = "normal"
        experiement_groups = [file_path_split[-2]]
    elif "sox" in file_path_split:
        experiement_type = "sox"
        experiement_groups = [file_path_split[-3], file_path_split[-2]]
    else:
        raise ValueError("file_path invalid, normal or sox expected: "+file_path)
    
    f = open(file_path,"r")
    line_id = 0
    for line in f:
        line_id += 1
        line = line.strip()
        #print(line)
        
        # Ignore empty lines
        if len(line) == 0:
            continue
        
        line = line.split(" ")
        line = [i for i in line if i != ""] # DBG to work with hand made errors of duplicate spaces
        #print(line)
        
        # Ignore first line
        if line[0] == "color":
            continue
            
        if line[0] == "#irradiation:":
            location = line[1].lower().index("gy")
            radiation_level = int(line[1][0:location])
            continue
            
        # Ignore comments
        if line[0][0] == "#":
            continue
            
        if radiation_level == -1:
            radiation_level = -1
            print(experiement_id+" radiation level not found before data, expected #irradiation: value")
            #raise ValueError(experiement_id+" radiation level not found before data, expected #irradiation: value")
        
        time = int(line[0])
        event = line[1]
        #print(line, " => ", end='')
        
        #print(time, cell_id, event, " ", end='')
        
        if event not in ["begin", "death", "end", "out", "div", "fusion"]:
            raise ValueError(experiement_id+" unexpected event value \""+ event +"\" at line "+str(line_id))
        
        if event == "begin":
            input_cells = []
            output_cells = line[2]
        
        elif event in ["death", "end", "out"]:
            input_cells = line[2]
            output_cells = []
        
        elif event == "div":
            input_cells = line[2]
            nb_cells = int(line[3])
            output_cells = []
            
            for i in range(nb_cells):
                output_cells.append(line[4+i])
            
        elif event == "fusion":
            input_cells = []
            output_cells = line[-1]
            
            for i in line[2:-2]:
                input_cells.append(i)
        
        data.append([time, event, input_cells, output_cells])
            
        #print()
    
    experiement_groups_str = '[%s]' % ', '.join(map(str, experiement_groups))
    experiement_id = experiement_type +"_"+ experiement_groups_str +"_"+ str(radiation_level) +"gy_"+  file_name[:-4]
        
    return experiement_type, experiement_groups, radiation_level, experiement_id, pd.DataFrame(data, columns=["time", "event", "input", "output"]).astype({'time': 'int'})

if DEBUG:
    print("convert_to_dataframe(example):")
    experiement_type, experiement_groups, radiation_level, experiement_id, df = convert_to_dataframe(EXAMPLE_TRANSCRIPTION_TXT_FILE)
    print("experiement type:", experiement_type)
    print("experiementgroups:", experiement_groups)
    print("radiation level:", radiation_level)
    print("experiement id:", experiement_id)
    display(df)

convert_to_dataframe(example):
experiement type: normal
experiementgroups: ['n2']
radiation level: 10
experiement id: normal_[n2]_10gy_2021n210Gy2B


,time,event,input,output
0,0,begin,[],C1
1,444,death,C1,[]
2,0,begin,[],C2
3,379,out,C2,[]
4,0,begin,[],C3
5,144,div,C3,"[C3.1, C3.2]"
6,202,fusion,"[C3.1, C3.2]",C3.3
7,218,death,C3.3,[]
8,0,begin,[],C4
9,175,death,C4,[]


In [10]:
formated_transcriptions = dict()

for dirname, _, filenames in os.walk(OUTPUT_TRANSCRIPTIONS_TXT_FOLDER):
    for filename in filenames:
        file_path = os.path.join(dirname, filename)
        
        if DEBUG: print("Extracting", file_path)
        try:
            experiement_type, experiement_groups, radiation_level, experiement_id, df = convert_to_dataframe(file_path)
            formated_transcriptions[experiement_id] = (experiement_type, experiement_groups, radiation_level, df)
        except ValueError as e:
            print("ERROR:", e)


Extracting videos_transcriptions_txt/normal/n1/160930.nd2u2514gyp1.txt
Extracting videos_transcriptions_txt/normal/n1/2020 11 04 0Gy A2B.txt
Extracting videos_transcriptions_txt/normal/n1/160930.nd2u2512gyp4.txt
Extracting videos_transcriptions_txt/normal/n1/160930.nd2u25110gyp1.txt
Extracting videos_transcriptions_txt/normal/n1/2020 11 04 0Gy 1A.txt
Extracting videos_transcriptions_txt/normal/n1/160930.nd2u2512gyp3.txt
Extracting videos_transcriptions_txt/normal/n1/160930.nd2u2518gyp4.txt
Extracting videos_transcriptions_txt/normal/n1/160930.nd2u2514gyp2.txt
Extracting videos_transcriptions_txt/normal/n1/2020 11 04 A3 0Gy.txt
Extracting videos_transcriptions_txt/normal/n1/160930.nd2u25115gyp4.txt
Extracting videos_transcriptions_txt/normal/n1/201104.nd2u2510GyA3A.txt
Extracting videos_transcriptions_txt/normal/n1/2020 11 04 0Gy A3A.txt
Extracting videos_transcriptions_txt/normal/n1/160930.nd2u25110gyp3.txt
Extracting videos_transcriptions_txt/normal/n1/2020 11 04 .0Gy B2A.txt
Extracti

Extracting videos_transcriptions_txt/normal/n2/2021 03 12 0Gy 12B.txt
Extracting videos_transcriptions_txt/normal/n2/2020 04 11 A4B  8gy.txt
Extracting videos_transcriptions_txt/normal/n2/2021N215GY5C.txt
Extracting videos_transcriptions_txt/normal/n2/2021n210Gy2BC.txt
Extracting videos_transcriptions_txt/normal/n2/2021n210Gy5C.txt
Extracting videos_transcriptions_txt/normal/n2/2021n210Gy5B.txt
Extracting videos_transcriptions_txt/normal/n2/2020-27-11-2Gy-B3B.txt
Extracting videos_transcriptions_txt/normal/n2/20200411B4Bu2518gy.txt
Extracting videos_transcriptions_txt/normal/n2/2021N215GY4B.txt
Extracting videos_transcriptions_txt/normal/n2/2020 04 11 B4A  8gy.txt
Extracting videos_transcriptions_txt/normal/n2/2020-27-11-2Gy-A4C.txt
Extracting videos_transcriptions_txt/normal/n2/2020 04 11 A4A  8gy.txt
Extracting videos_transcriptions_txt/normal/n2/2020 04 11 C3B  8gy.txt
Extracting videos_transcriptions_txt/normal/n2/2021n210Gy4C.txt
Extracting videos_transcriptions_txt/normal/n2/2021

# 2. Data cleaning

In [11]:
# Properties:
# - All event time >= 0
# - All and only begin event are a time 0 (warning only)
# - Each cell appear in exactly one event output (produced only once)
# - Event input cells must be output of a previous event (already produced by a begin/div/fusion) TODO or same frame
# - Event input cells cannot be input of other events (cell is consumed by first death/end/out/div/fusion)
# - All output cells are input of one ending event (terminated by death/end/out/div/fusion)

for exp_id, (experiement_type, experiement_groups, radiation_level, df) in formated_transcriptions.items():
    #print(exp_id)
    #print(radiation_level)
    #print(df)
    error_msg_start = "ERROR: Experiement "+exp_id+" on "
    warning_msg_start = "WARNING: Experiement "+exp_id+" on "
    
    # Extract all events ordered by time
    all_events = [(t,e,i,o) for (t,e,i,o) in df.values]
    all_events.sort()
    
    # 1) All event time >= 0
    for event_id, (time, event, input, output) in enumerate(all_events):
        if time < 0:
            print(error_msg_start,"(", time, event, input, output, "):", "time cannot be < 0")
    
    # 2) All and only begin event are a time 0
    for event_id, (time, event, input, output) in enumerate(all_events):
        if event == "begin" and time != 0:
            print(warning_msg_start,"(", time, event, input, output, "):", "begin is usually 0")
        if event != "begin" and time == 0:
            print(error_msg_start,"(", time, event, input, output, "):", "only begin are allowed at time 0")
        
    # 2) Each cell appear in exactly one event output (produced only once)
    started = dict()
    for event_id, (time, event, input, output) in enumerate(all_events):
        # Extract start time of each cell
        if not isinstance(output, list):
            output = [output]
        for cell in output:
            if cell in started:
                print(error_msg_start,"(", time, event, input, output, "):", cell, " already existing from", started[cell])
            else:
                started[cell] = (time, event, input, output)
        
    # 3) Event input cells must be output of a previous event (already produced by a begin/div/fusion)
    for event_id, (time, event, input, output) in enumerate(all_events):
        if not isinstance(input, list):
            input = [input]
        for cell in input:
            if cell not in started:
                print(error_msg_start,"(", time, event, input, output,"):", cell, "does not exist yet ", end='')
                print("(",cell,"is never an output of any begin/div/fusion)")
            else:
                if started[cell][0] >= time:
                    if started[cell][0] == time:
                        print(warning_msg_start,"(", time, event, input, output,"):", cell, "does not exist yet ", end='')
                    else:
                        print(error_msg_start,"(", time, event, input, output,"):", cell, "does not exist yet ", end='')
                    print("(created by",started[cell],")")
            
    
    # 4) Event input cells cannot be input of other events (cell is consumed by first death/end/out/div/fusion)
    final_event_labels = ["death", "out", "end", "div", "fusion"]
    final_events = df[(df["event"].isin(final_event_labels))][["time", "event", "input"]]
    final_events = [(t,e,c) for (t,e,c) in final_events.values]
    final_events.sort()
    
    ended = dict()
    for event_id, (time, event, input, output) in enumerate(all_events):
        # Extract start time of each cell
        if event in final_event_labels:
            if not isinstance(input, list):
                input = [input]
            for cell in input:
                if cell in ended:
                    print(error_msg_start,"(",time,event,input,output,"):", cell,"is already terminated by", ended[cell])
                else:
                    ended[cell] = (time, event, input, output)
                    
    # 5) All output cells are input of one ending event (terminated by death/end/out/div/fusion)
    for cell_id in started:
        if cell_id not in ended:
            print(error_msg_start, cell_id,"never seen as input of end/death/out/div/fusion")
    

- Correct properties checked
    - All event time >= 0 OK
    - All and only begin event are a time 0 nope some after
    - Each cell appear in exactly one event output (produced only once) OK
    - Each cell appear in exactly one event input (consumed only once) OK
    - Event input cells must be output of a previous event (already produced by a begin/div/fusion at previous time) or same frame

In [12]:
!tree

.
├── glioblastoma_data
│   ├── extracted_data
│   │   └── cell_behaviors
│   │       ├── examples
│   │       │   ├── example_videos_transcriptions_csv.csv
│   │       │   └── example_videos_transcriptions_txt.txt
│   │       ├── README.txt
│   │       ├── videos_transcriptions_csv
│   │       │   ├── normal
│   │       │   │   ├── n1
│   │       │   │   │   ├── 0gy
│   │       │   │   │   │   ├── normal_[n1]_0gy_201104.nd2u2510Gy1A.csv
│   │       │   │   │   │   ├── normal_[n1]_0gy_201104.nd2u2510GyA3A.csv
│   │       │   │   │   │   ├── normal_[n1]_0gy_2020 11 04 0Gy 1A.csv
│   │       │   │   │   │   ├── normal_[n1]_0gy_2020 11 04 0Gy A2B.csv
│   │       │   │   │   │   ├── normal_[n1]_0gy_2020 11 04 0Gy A3A.csv
│   │       │   │   │   │   └── normal_[n1]_0gy_2020 11 04 .0Gy B2A.csv
│   │       │   │   │   ├── 10gy
│   │       │   │   │   │   ├── normal_[n1]_10gy_160930.nd2u25110gyp1.csv
│   │       │   │   │   │   ├── normal_[n1]_10gy_160930.nd2u25110gyp2.csv
│   │       │   │   

In [13]:
for exp_id, (experiement_type, experiement_groups, radiation_level, df) in formated_transcriptions.items():
    group_folders = "/".join(experiement_groups)
    csv_path = "videos_transcriptions_csv/"+ experiement_type +"/"+ group_folders +"/"+ str(radiation_level)+"gy"
    try:
        os.mkdir(csv_path)
    except OSError as error:
        pass
    csv_path += "/"+ exp_id+".csv"
    df.to_csv(csv_path, index=False)
    
!zip -r "videos_transcriptions_csv.zip" videos_transcriptions_csv

  adding: videos_transcriptions_csv/ (stored 0%)
  adding: videos_transcriptions_csv/normal/ (stored 0%)
  adding: videos_transcriptions_csv/normal/n1/ (stored 0%)
  adding: videos_transcriptions_csv/normal/n1/4gy/ (stored 0%)
  adding: videos_transcriptions_csv/normal/n1/4gy/normal_[n1]_4gy_160930.nd2u2514gyp3.csv (deflated 50%)
  adding: videos_transcriptions_csv/normal/n1/4gy/normal_[n1]_4gy_160930.nd2u2514gyp4.csv (deflated 69%)
  adding: videos_transcriptions_csv/normal/n1/4gy/normal_[n1]_4gy_160930.nd2u2514gyp2.csv (deflated 70%)
  adding: videos_transcriptions_csv/normal/n1/4gy/normal_[n1]_4gy_160930.nd2u2514gyp1.csv (deflated 64%)
  adding: videos_transcriptions_csv/normal/n1/0gy/ (stored 0%)
  adding: videos_transcriptions_csv/normal/n1/0gy/normal_[n1]_0gy_201104.nd2u2510GyA3A.csv (deflated 78%)
  adding: videos_transcriptions_csv/normal/n1/0gy/normal_[n1]_0gy_2020 11 04 0Gy 1A.csv (deflated 78%)
  adding: videos_transcriptions_csv/normal/n1/0gy/normal_[n1]_0gy_201104.nd2u2510

In [14]:
FileLink('videos_transcriptions_txt.zip')

/home/keijuro/Bureau/epicea/videos_transcriptions_txt.zip

In [15]:
FileLink('videos_transcriptions_csv.zip')

/home/keijuro/Bureau/epicea/videos_transcriptions_csv.zip